In [1]:
%matplotlib inline
#%matplotlib widget


In [2]:
#############imports######################################################################
import cv2 as cv2
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import trackpy as tp
from ipywidgets import HBox, Textarea, interact


In [24]:
expSumFileName ='C:\\Users\\User\\Documents\\Eden\\locate_git_code\\packaged\\expSumPack21.csv'
expSum = pd.read_csv(expSumFileName,index_col=0)
expnum=26
passive_path='C:\\Users\\User\\Documents\\Eden\\locate_git_code\\packaged'
expSum=expSum.iloc[expnum]
#for index,row in expSum.iterrows():
PassiveLocatingFileName=expSum.locatingFileNamePack
print(PassiveLocatingFileName)
PassiveLocating=pd.read_csv(passive_path+'\\'+PassiveLocatingFileName,index_col=0)
#first_passive_x=PassiveLocating.x[0]
#first_passive_y=PassiveLocating.y[0]
PassiveLocating.set_index('frame',inplace=True)
#print(PassiveLocating)
#phi=np.degrees(np.arctan2((PassiveLocating.y-first_passive_y),(PassiveLocating.x-first_passive_x)))
velocity=PassiveLocating[['x','y']].diff()*25
velocity['absolute']=np.sqrt(np.power(velocity.x,2)+np.power(velocity.y,2))
print(velocity)

#phi=np.tan(velocity.y/velocity.x)
#print(phi)

fronter_N021_D028_C0490.MP4_loc.csv
            x     y   absolute
frame                         
1.0       NaN   NaN        NaN
2.0       0.0   0.0   0.000000
3.0     -25.0 -25.0  35.355339
4.0       0.0  50.0  50.000000
5.0      50.0 -50.0  70.710678
...       ...   ...        ...
31040.0 -25.0 -25.0  35.355339
31041.0  75.0   0.0  75.000000
31042.0 -25.0   0.0  25.000000
31043.0  50.0 -25.0  55.901699
31044.0 -25.0  25.0  35.355339

[31003 rows x 3 columns]


In [25]:
expSumbotFileName='C:\\Users\\User\\Documents\\Eden\\locate_git_code\\packaged\\botsTracking\\expSumPack21.csv'
expSumbot = pd.read_csv(expSumbotFileName,index_col=0)
passive_bot_path='C:\\Users\\User\\Documents\\Eden\\locate_git_code\\packaged\\botsTracking'
expSumbot=expSumbot.iloc[expnum]
BotLocatingFileName=expSumbot.botsLocatingFileNamePack
BotLocating=pd.read_csv(passive_bot_path+'\\'+BotLocatingFileName,index_col=0)
#close_threshold=55**2
close_threshold=140**2
#too_close_thresh=18**2
too_close_thresh=100**2
r_vector_time=pd.DataFrame()
for index, row in PassiveLocating.iterrows():
    if index%1000==0:
        print(index)
    relevant_passive_data=row
    relevant_bots_data=BotLocating[(BotLocating['frame']==index)]
    bots_x=relevant_bots_data.x
    bots_y=relevant_bots_data.y
    distance=np.array(np.power((row.x-bots_x),2)+np.power((row.y-bots_y),2))
    relevant_bots_data_distance=pd.DataFrame({'frame':relevant_bots_data['frame'],'distance_sq':distance,
                                              'x':relevant_bots_data.x,'y':relevant_bots_data.y})
    #BotLocating.loc[(BotLocating['frame']==index),'distance_sq']=distance
    cond=(relevant_bots_data_distance.distance_sq<=close_threshold)&(relevant_bots_data_distance.distance_sq>too_close_thresh)
    relevant_bots_dataCG=relevant_bots_data_distance[cond]
    r_vector_x=np.mean(row.x-relevant_bots_dataCG.x)
    
    r_vector_y=np.mean(row.y-relevant_bots_dataCG.y)
    r_vector_time_row=pd.DataFrame()
    r_vector_time_row['frame']=np.array([index])
    r_vector_time_row['x']=np.array([r_vector_x])
    r_vector_time_row['y']=np.array([r_vector_y])
    r_vector_time_row['absolute']=np.array([np.sqrt(np.power(r_vector_x,2)+np.power(r_vector_y,2))])
    r_vector_time_row['phi']=np.array([np.degrees(np.arctan2(r_vector_y,r_vector_x))])
    r_vector_time=pd.concat([r_vector_time,r_vector_time_row],axis=0,ignore_index=True)
print(r_vector_time)

1000.0
2000.0
3000.0
4000.0
5000.0
6000.0
7000.0
8000.0
9000.0
10000.0
11000.0
12000.0
13000.0
14000.0
15000.0
16000.0
17000.0
18000.0
19000.0
20000.0
21000.0
22000.0
23000.0
24000.0
25000.0
26000.0
27000.0
28000.0
29000.0
30000.0
31000.0
         frame           x          y    absolute         phi
0          1.0   -1.571429  -1.761905    2.360868 -131.729512
1          2.0   -1.380952  -1.523810    2.056459 -132.184443
2          3.0   -2.619048  -2.523810    3.637173 -136.060912
3          4.0   -2.333333  -0.190476    2.341095 -175.333142
4          5.0   -0.857143  -2.619048    2.755740 -108.121860
...        ...         ...        ...         ...         ...
30998  31040.0  107.000000  60.000000  122.674366   29.281427
30999  31041.0  110.000000  60.000000  125.299641   28.610460
31000  31042.0  109.000000  59.000000  123.943536   28.425992
31001  31043.0  110.000000  62.000000  126.269553   29.407189
31002  31044.0  110.000000  60.000000  125.299641   28.610460

[31003 rows x 5 

In [26]:
r_vector_time.to_csv('C:\\Users\\User\\Documents\\Eden\\locate_git_code\\phi_test.csv')

In [36]:
p=(r_vector_time.x*velocity.x+r_vector_time.y*velocity.y)/(r_vector_time.absolute*velocity.absolute)
p.fillna(0,inplace=True)
print(p)
p.to_csv('C:\\Users\\User\\Documents\\Eden\\locate_git_code\\p_test.csv')
x=r_vector_time.x*velocity.x
x.to_csv('C:\\Users\\User\\Documents\\Eden\\locate_git_code\\p_test_x.csv')
y=r_vector_time.y*velocity.y
y.to_csv('C:\\Users\\User\\Documents\\Eden\\locate_git_code\\p_test_y.csv')
absolute=r_vector_time.absolute*velocity.absolute
absolute.to_csv('C:\\Users\\User\\Documents\\Eden\\locate_git_code\\p_test_abs.csv')

0.0        0.000000
1.0        0.000000
2.0        0.000000
3.0        0.762294
4.0       -0.950397
             ...   
31040.0    0.000000
31041.0    0.000000
31042.0    0.000000
31043.0    0.000000
31044.0    0.000000
Length: 31045, dtype: float64


In [33]:
test=np.arange(len(y))//(25)
print(np.arange(len(y))//(25))
test.tofile('C:\\Users\\User\\Documents\\Eden\\locate_git_code\\p_test_arrange.csv')

[   0    0    0 ... 1241 1241 1241]
